In [2]:
import sys, os
import rasterio

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.wkt import loads

sys.path.insert(0, "../src")

import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.dataMisc as dMisc

%load_ext autoreload
%autoreload 2

In [3]:
# Local/input files
iso3 = 'KHM'
out_folder = "c:/WBG/Work/KHM_Energy/data"
wsf_file = os.path.join(out_folder, "WSF", "wsf.tif")
ghsl_file = os.path.join(out_folder, "GHSL", "ghsl.tif")
overture_buildings = os.path.join(out_folder, "overture", "overture_download_2024_03_29.csv")
overture_raster = os.path.join(out_folder, "overture", "overture_download_2024_03_29.tif")
overture_raster_points = os.path.join(out_folder, "overture", "overture_download_2024_03_29_points.tif")

google_buildings = os.path.join(out_folder, "Google_Buildings", "311_buildings.csv")
google_raster = os.path.join(out_folder, "Google_Buildings", "311_buildings.tif")
google_raster_points = os.path.join(out_folder, "Google_Buildings", "311_buildings_points.tif")
for file in [wsf_file, ghsl_file]:
    if not os.path.exists(os.path.dirname(file)):
        os.makedirs(os.path.dirname(file))

# get country extent from geopandas
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
country = world[world.iso_a3 == iso3]

C:\Users\WB411133\AppData\Local\Temp\ipykernel_26056\3528700091.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world_filepath = gpd.datasets.get_path('naturalearth_lowres')


In [ ]:
""" Not working with World Bank Firewall
# Download the WSF data
if not os.path.exists(wsf_file):
    print("Downloading WSF data")
    wsf_data, wsf_profile = dMisc.download_WSF(country, out_file = wsf_file)

wsf_r = rasterio.open(wsf_file)
"""
print("FUBAR")

In [13]:
#Clip GHSL using local files
local_version = r"J:\Data\GLOBAL\GHSL\Built\GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0.tif"
if not os.path.exists(ghsl_file):
    ghsl_raster = rasterio.open(local_version)
    data, profile = rMisc.clipRaster(ghsl_raster, country)
    with rasterio.open(ghsl_file, 'w', **profile) as dst:
        dst.write(data)
ghsl_r = rasterio.open(ghsl_file)

In [4]:
# read in an process Google buildings
google_buildings = pd.read_csv(google_buildings)
google_buildings['geometry'] = google_buildings['geometry'].apply(loads)
inG = gpd.GeoDataFrame(google_buildings, geometry = 'geometry', crs=4326)
inG.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,13.522077,101.216031,92.9937,0.8560,"POLYGON((101.216096132875 13.5220590997704, 10...",7P53G6C8+RCJ5
1,10.941230,105.193386,26.2096,0.7669,"POLYGON((105.193413459308 10.9412118246212, 10...",7P27W5RV+F9R7
2,11.055472,105.636249,122.0114,0.7124,"POLYGON((105.6362994 11.0554098, 105.6363091 1...",7P373J4P+5FQX
3,10.714396,103.813111,75.0603,0.7619,"POLYGON((103.813151906226 10.7143466112636, 10...",7P25PR77+Q65W
4,14.213793,101.339616,8.7935,0.7022,"POLYGON((101.339628618998 14.2137778129493, 10...",7P63687Q+GR9P


In [9]:
# attempt to rasterrize the buildings as polygons
if not os.path.exists(google_raster):
    rasterized_buildings = rMisc.rasterizeDataFrame(inG, '', templateRaster=ghsl_file, mergeAlg="ADD", re_proj=True, nodata=0.)
    with rasterio.open(google_raster, 'w', **rasterized_buildings['meta']) as dst:
        dst.write_band(1, rasterized_buildings['vals'])
google_r = rasterio.open(google_raster)

# attempt to rasterrize the buildings as points
if not os.path.exists(google_raster_points):
    inG_points = inG.copy()
    inG_points['geometry'] = inG_points['geometry'].centroid
    rasterized_buildings = rMisc.rasterizeDataFrame(inG_points, '', templateRaster=ghsl_file, mergeAlg="ADD", re_proj=True, nodata=0.)
    with rasterio.open(google_raster_points, 'w', **rasterized_buildings['meta']) as dst:
        dst.write_band(1, rasterized_buildings['vals'])
google_r_points = rasterio.open(google_raster_points) 

C:\Users\WB411133\AppData\Local\Temp\ipykernel_26056\3879913162.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inG_points['geometry'] = inG_points['geometry'].centroid


In [15]:
# Compare rasterized buildings with built area dataset
### Open both datasets and threshold them to get built area data
g_thresh = 1
ghsl_thresh = 3000

g_data = google_r_points.read(1)
g_data = (g_data > g_thresh).astype('uint8')

ghsl_data = ghsl_r.read(1)
ghsl_data = (ghsl_data > ghsl_thresh).astype('uint8') * 10

combo_data = g_data + ghsl_data

# Write out the combined data
out_file = os.path.join(out_folder, "google_vs_ghsl.tif")
if not os.path.exists(out_file):
    meta = overture_r_points.meta.copy()
    meta.update(dtype=rasterio.uint8, nodata=0)
    with rasterio.open(out_file, 'w', **meta) as out_raster:
        out_raster.write_band(1, combo_data)

In [ ]:
# read in and process Overture buildings
ob = pd.read_csv(overture_buildings)
ob_geoms = ob['wkt'].apply(loads)
inB = gpd.GeoDataFrame(ob, geometry=ob_geoms, crs=4326)
inB.head()


In [ ]:
# attempt to rasterrize the buildings as polygons
if not os.path.exists(overture_raster):
    rasterized_buildings = rMisc.rasterizeDataFrame(inB, templateRaster=ghsl_file, mergeAlg="ADD", re_proj=True, nodata=0.)
    with rasterio.open(overture_raster, 'w', **rasterized_buildings['meta']) as dst:
        dst.write_band(1, rasterized_buildings['vals'])
overture_r = rasterio.open(overture_raster)

In [ ]:
# attempt to rasterrize the buildings as points
if not os.path.exists(overture_raster_points):
    inB_points = inB.copy()
    inB_points['geometry'] = inB_points['geometry'].centroid
    rasterized_buildings = rMisc.rasterizeDataFrame(inB_points, templateRaster=ghsl_file, mergeAlg="ADD", re_proj=True, nodata=0.)
    with rasterio.open(overture_raster_points, 'w', **rasterized_buildings['meta']) as dst:
        dst.write_band(1, rasterized_buildings['vals'])
overture_r_points = rasterio.open(overture_raster_points) 

In [ ]:
# Compare rasterized buildings with built area dataset
### Open both datasets and threshold them to get built area data
o_thresh = 1
ghsl_thresh = 3000

o_data = overture_r_points.read(1)
o_data = (o_data > o_thresh).astype('uint8')

ghsl_data = ghsl_r.read(1)
ghsl_data = (ghsl_data > ghsl_thresh).astype('uint8') * 10

combo_data = o_data + ghsl_data

# Write out the combined data
out_file = os.path.join(out_folder, "overture_vs_ghsl.tif")
if not os.path.exists(out_file):
    meta = overture_r_points.meta.copy()
    meta.update(dtype=rasterio.uint8, nodata=0)
    with rasterio.open(out_file, 'w', **meta) as out_raster:
        out_raster.write_band(1, combo_data)
